In [1]:
from urllib.request import urlopen
import pandas as pd
import json
from random import randint

pd.set_option('expand_frame_repr', False) # 列不换行
pd.set_option('display.max_rows', 5000) # 最多显示行数

# 返回数据网址
# 获取代码网址 http://web.ifzq.gtimg.cn/appstock/app/fqkline/get?_var=kline_dayqfq&param=sz000001,day,,,10,qfq&r=0.5643184591626897
# 正常网址 http://stockhtm.finance.qq.com/sstock/ggcx/000001.shtml
# 
# 关键参数sz000001 10 day 16位随机数

stock_code = 'sz000001' #股票代码
k_type = 'day' # 日线 day week month
num = 10 # 获取k线根数

'''
参数说明：
可返回股票代码，如sz000001
可返回指数，如sh000001
可返回ETF，如sh5105000
'''

'\n参数说明：\n可返回股票代码，如sz000001\n可返回指数，如sh000001\n可返回ETF，如sh5105000\n'

### 创建16位随机数

In [2]:
def _random(n=16):
    """
    创建n位随机整数
    """
    start = 10**(n-1)
    end = (10**n)-1
    return str(randint(start, end))

### 构建网址

In [3]:
url = 'http://web.ifzq.gtimg.cn/appstock/app/fqkline/get?_var=kline_%sqfq&param=%s,%s,,,%s,qfq&r=0.%s'
url = url % (k_type, stock_code, k_type, num, _random())

print(url)

http://web.ifzq.gtimg.cn/appstock/app/fqkline/get?_var=kline_dayqfq&param=sz000001,day,,,10,qfq&r=0.7071218121927124


### 抓取数据

In [4]:
content = urlopen(url).read().decode('gbk')
content = content.split('=', maxsplit=1)[-1]
content = json.loads(content)

print(type(content))
print(content)

<class 'dict'>
{'code': 0, 'msg': '', 'data': {'sz000001': {'qfqday': [['2019-05-09', '12.520', '12.160', '12.580', '12.050', '1557151.000'], ['2019-05-10', '12.340', '12.680', '12.750', '12.100', '1192399.000'], ['2019-05-13', '12.330', '12.300', '12.540', '12.230', '741917.000'], ['2019-05-14', '12.200', '12.490', '12.750', '12.160', '1182598.000'], ['2019-05-15', '12.580', '12.920', '13.110', '12.570', '1103988.000'], ['2019-05-16', '12.930', '12.850', '12.990', '12.780', '634901.000'], ['2019-05-17', '12.920', '12.440', '12.930', '12.360', '965000.000'], ['2019-05-20', '12.350', '12.380', '12.540', '12.250', '786435.000'], ['2019-05-21', '12.400', '12.560', '12.730', '12.360', '731210.000'], ['2019-05-22', '12.570', '12.400', '12.570', '12.320', '506230.000']], 'qt': {'sz000001': ['51', '平安银行', '000001', '12.40', '12.56', '12.57', '506230', '211240', '294963', '12.40', '906', '12.39', '4680', '12.38', '482', '12.37', '938', '12.36', '2735', '12.41', '843', '12.42', '256', '12.43', 

### 整理数据

In [5]:
k_data = content['data'][stock_code]
print(k_data)

if k_type in k_data:
    k_data = k_data[k_type]
elif 'qfq' + k_type in k_data:
    k_data = k_data['qfq' + k_type]
else:
    raise ValueError('已知key不存在')
    
df = pd.DataFrame(k_data)
print(df)

{'qfqday': [['2019-05-09', '12.520', '12.160', '12.580', '12.050', '1557151.000'], ['2019-05-10', '12.340', '12.680', '12.750', '12.100', '1192399.000'], ['2019-05-13', '12.330', '12.300', '12.540', '12.230', '741917.000'], ['2019-05-14', '12.200', '12.490', '12.750', '12.160', '1182598.000'], ['2019-05-15', '12.580', '12.920', '13.110', '12.570', '1103988.000'], ['2019-05-16', '12.930', '12.850', '12.990', '12.780', '634901.000'], ['2019-05-17', '12.920', '12.440', '12.930', '12.360', '965000.000'], ['2019-05-20', '12.350', '12.380', '12.540', '12.250', '786435.000'], ['2019-05-21', '12.400', '12.560', '12.730', '12.360', '731210.000'], ['2019-05-22', '12.570', '12.400', '12.570', '12.320', '506230.000']], 'qt': {'sz000001': ['51', '平安银行', '000001', '12.40', '12.56', '12.57', '506230', '211240', '294963', '12.40', '906', '12.39', '4680', '12.38', '482', '12.37', '938', '12.36', '2735', '12.41', '843', '12.42', '256', '12.43', '466', '12.44', '584', '12.45', '686', '15:00:04/12.40/5243

In [6]:
rename_dict ={0:'candle_end_time', 1:'open', 2:'close', 3:'high', 4:'low', 5:'amount', 6:'info'}
# amount 单位为手
df.rename(columns=rename_dict, inplace=True)

if 'info' not in df:
    df['info'] = None
df = df[['candle_end_time', 'open', 'high', 'low', 'close', 'amount', 'info']]

print(df)

  candle_end_time    open    high     low   close       amount  info
0      2019-05-09  12.520  12.580  12.050  12.160  1557151.000  None
1      2019-05-10  12.340  12.750  12.100  12.680  1192399.000  None
2      2019-05-13  12.330  12.540  12.230  12.300   741917.000  None
3      2019-05-14  12.200  12.750  12.160  12.490  1182598.000  None
4      2019-05-15  12.580  13.110  12.570  12.920  1103988.000  None
5      2019-05-16  12.930  12.990  12.780  12.850   634901.000  None
6      2019-05-17  12.920  12.930  12.360  12.440   965000.000  None
7      2019-05-20  12.350  12.540  12.250  12.380   786435.000  None
8      2019-05-21  12.400  12.730  12.360  12.560   731210.000  None
9      2019-05-22  12.570  12.570  12.320  12.400   506230.000  None


### 备注

info中存储除权信息，如果有会显示

股票日线数据不超过640根，指数，ETF不限
